# ChE 212: Homework 2 Problem 1

# Problem Statement:

A binary mixture of 40 mol% vinylacetylene and 60 mol% toluene is to be flashed at 1 atm pressure and 60C. The Vapor-Liquid Equilibrium (VLE) is adequately represented by Raoult's Law, with the Antoine coefficients for vapor pressures given in the table below.

a) Solve the Rachford-Rice equations for the fraction of the feed that is vapor ($\phi$) and for the composition of the vapor and liquid streams leaving the flash vessel.

b) Repeat the calculations at the same T and P, but with feed compositions of 30 mol% and 50 mol% vinylacetylene.

Antoine Coefficient Data (P in mmHg, T in C)

|   | Toluene | Vinylacetylene |
|---|---------|----------------|
| A | 6.95087 | 7.02515        |
| B | 1342.31 | 999.110        |
| C | 219.187 | 235.817        |

## Let's import Python packages we might need

In [13]:
import numpy as np
import matplotlib.pyplot as plt

## Set up Rachford-Rice Equation

$$\begin{equation}
    Fz_i = Vy_i + Lx_i \label{Fzi} \\
\end{equation}$$
$$\begin{equation}
    z_i = \frac{V}{F}y_i + \frac{L}{F}x_i \label{zi}
\end{equation}$$

Define:
$$\begin{equation}
    V = \phi F \label{V} \\
\end{equation}$$
$$\begin{equation}
    \phi = \frac{V}{F} \label{phi}
\end{equation}$$

And:
$$\begin{equation}
    L =F-V \label{L} \\
\end{equation}$$
$$\begin{equation}
    L=F-\phi F = F(1-\phi) \label{Lsub}
\end{equation}$$

Plug in Equation \ref{phi}
$$\begin{equation}
    \frac{L}{F} = (1-\phi) \label{Lsub2}
\end{equation}$$

And: 

$$\begin{equation}
    z_i = \phi y_i + (1-\phi)x_i \label{ziphi}
\end{equation}$$

We know Raoult's Law:

$$\begin{equation}
    Py_i = P_i^{sat}x_i \label{Psat}
\end{equation}$$

Rearrange for $y_i$:
$$\begin{equation}
    y_i = \frac{P_i^{sat}}{P}x_i = K_ix_i \label{Ki}
\end{equation}$$
where $K_i$ is the equilibrium value - which is constant for ideal mixtures at a single T and P

Substitute Equation $\ref{eq:Ki}$ into Equation $\ref{eq:ziphi}$

$$\begin{equation}
    z_i = \phi K_ix_i + (1-\phi)x_i \label{ziKi}
\end{equation}$$

$$\begin{equation}
    z_i = x_i(\phi K_i + (1-\phi)) = x_i(1 + \phi(K_i-1)) \label{ziKi2}
\end{equation}$$

Now:

$$\begin{equation}
    x_i = \frac{z_i}{1+\phi(K_i-1)} \label{xiziKi}
\end{equation}$$

$$\begin{equation}
    y_i = \frac{K_iz_i}{1+\phi(K_i-1)} \label{yiziKi}
\end{equation}$$

And we know:
$$\begin{equation}
    \sum y_i - \sum x_i = 0 \label{sum}
\end{equation}$$

Plugging in everything:
$$\begin{equation}
    \sum_{i} \frac{z_i(K_i - 1)}{1+\phi(K_i - 1)} = 0 \label{RRE}
\end{equation}$$

We get the Rachford-Rice Equation!

## What Variables Do We Need To Define?

Let "1" be Toluene and "2" be Vinylacetylene

In [110]:
## Temperature and Pressure
T = 60 #C
P = 1 #atm

## Antoine Coefficients
A_tol = 6.95087
B_tol = 1342.31
C_tol = 219.2
A_va = 7.02515
B_va = 999.11
C_va = 235.817

## Entering Mol Fractions
z1 = .6
z2 = 1 - z1

## A basis
Basis = 1000 ##kmol/hr

## We can solve for $P^{sat}$, $K_i$, and $F$ using our equations above.

In [87]:
## Saturated Pressures using Antoine Equation

Psat_tol_mmHg = 10**(A_tol - B_tol/(T+C_tol)) ## mmHg
Psat_tol = Psat_tol_mmHg/760 ## atm

Psat_va_mmHg = 10**(A_va - B_va/(T+C_va)) ## mmHg
Psat_va = Psat_va_mmHg/760 ## atm

## Ki = Psat/P

K1 = Psat_tol/P ## unitless
K2 = Psat_va/P ## unitless

## F

F_tol = z_tol * Basis ## kmol/hr
F_va = z_va * Basis ## kmol/hr

## What's $\phi$? We have to find it using the Rachford-Rice Equation and iteration

We need to make
$$\frac{z_1(K_1-1)}{(1+\phi(K_1-1))} = \frac{z_2(K_2-1)}{(1+\phi(K_2-1))}$$

## We use a similar approach to Homework 1 Problem 3

## Let's define a function that will help solve for $\phi$. Here's we'll move the $z_2$ equation to the other side to solve

$$\frac{z_1(K_1-1)}{(1+\phi(K_1-1))} + \frac{z_2(K_2-1)}{(1+\phi(K_2-1))} = 0$$

## A minimization problem!

In [88]:
def phi_solver(phi,z1,z2):
    z1eq = (z1*(K1-1))/(1+(phi*(K1-1)))
    z2eq = (z2*(K2-1))/(1+(phi*(K2-1)))
    diff = abs(z1eq + z2eq) ## this is what we want to minimize
    return diff

## We need to define an initial guess array. We don't know what $\phi$ might be, so we might make a guess that $\phi$ is between 0 and 1. We choose 1000 increments between 0 and 1 to be evaluated
#### Note: 1000 increments is arbitrary. One can choose any positive integer!

In [89]:
phi_guess = np.linspace(0,1,1000) # phi can only be between 0 and 1

## Let's solve for $\phi$!

In [90]:
min_diff = 1000 ## we define a variable min_diff that sets a baseline our difference comparison

for i in range(0,len(phi_guess)):
    diff_out = phi_solver(phi_guess[i],z1,z2) ## using the function above pull out phi
    if diff_out < min_diff: ## if we reach a new minimum (less than the previous min_diff)
        min_diff = diff_out ## we establish whatever difference from the function output as the new min_diff
        phi = phi_guess[i] ## this pulls out the phi at that minimum difference


## rounding phi to not have so many decimals
phi = round(phi,3) ## three decimals
print(f"Phi: {phi}") ## this prints the phi that satisfies the Rachford Rice Equation

Phi: 0.365


## Great, we have $\phi$! Now we can solve for $x_i$ and $y_i$

In [91]:
x1 = z1/(1+(phi*(K1-1)))
x2 = 1 - x1 ## because mol fractions have to add up to 1
y1 = K1*x1
y2 = 1 - y1

## Let's round to two decimals so we don't have really long numbers
x1 = round(x1,2)
x2 = round(x2,2)
y1 = round(y1,2)
y2 = round(y2,2)

In [92]:
print(f"Toluene x: {x1}, y: {y1}")
print(f"Vinylacetylene x: {x2}, y: {y2}")

Toluene x: 0.85, y: 0.16
Vinylacetylene x: 0.15, y: 0.84


## So that was part a. Let's move onto part b - which isn't too bad because we already did the hard work! Everything is the same but our $z_1$ and $z_2$

In [109]:
## when z_va = .3

## changing the z values
z1 = .7
z2 = 1 - z1 

## solve for phi
min_diff = 1000 ## have to reset this!

for i in range(0,len(phi_guess)):
    diff_out = phi_solver(phi_guess[i],z1,z2) ## using the function above pull out phi
    if diff_out < min_diff: ## if we reach a new minimum (less than the previous min_diff)
        min_diff = diff_out ## we establish whatever difference from the function output as the new min_diff
        phi = phi_guess[i] ## this pulls out the phi at that minimum difference
phi = round(phi,3)
print(f"Phi: {phi}")

## calculate x1, x2, y1, y2
x1 = z1/(1+(phi*(K1-1)))
x2 = 1 - x1 ## because mol fractions have to add up to 1
y1 = K1*x1
y2 = 1 - y1

## Let's round to two decimals so we don't have really long numbers
x1 = round(x1,2)
x2 = round(x2,2)
y1 = round(y1,2)
y2 = round(y2,2)

## print answer
print(f"Toluene x: {x1}, y: {y1}")
print(f"Vinylacetylene x: {x2}, y: {y2}")

Phi: 0.223
Toluene x: 0.86, y: 0.16
Vinylacetylene x: 0.14, y: 0.84


In [111]:
## when z_va = .3

## changing the z values
z1 = .5
z2 = 1 - z1 

## solve for phi
min_diff = 1000 ## have to reset this!

for i in range(0,len(phi_guess)):
    diff_out = phi_solver(phi_guess[i],z1,z2) ## using the function above pull out phi
    if diff_out < min_diff: ## if we reach a new minimum (less than the previous min_diff)
        min_diff = diff_out ## we establish whatever difference from the function output as the new min_diff
        phi = phi_guess[i] ## this pulls out the phi at that minimum difference
phi = round(phi,3)
print(f"Phi: {phi}")

## calculate x1, x2, y1, y2
x1 = z1/(1+(phi*(K1-1)))
x2 = 1 - x1 ## because mol fractions have to add up to 1
y1 = K1*x1
y2 = 1 - y1

## Let's round to two decimals so we don't have really long numbers
x1 = round(x1,2)
x2 = round(x2,2)
y1 = round(y1,2)
y2 = round(y2,2)

## print answer
print(f"Toluene x: {x1}, y: {y1}")
print(f"Vinylacetylene x: {x2}, y: {y2}")

Phi: 0.509
Toluene x: 0.86, y: 0.16
Vinylacetylene x: 0.14, y: 0.84


## Notice anything interesting about your answers?

## They should be the same! Why? Because the mixture exits in equilibrium, which is always the same at a given temperature and pressure!